In [1]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
vp_week = pd.read_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_año.xlsx") #ventas perdidas
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet") #movimiento  facturas
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet") #Costos unitarios

In [2]:
df.head()

,origin,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv
0,BD9/OUT/07176,56400.0,1625201,3.0,[BLS10050125] GS050 FILTRO ACEITE BALDWIN - FO...,FCAL5717,2025-10-01 14:07:04,FCAL5717,440706,SUCURSAL CALI,CLAUDIA ORTIZ,SUCURSALES / CALI / MOSTRADOR CALI CREDITO,35670,[CO9003546375] HIDALGO E HIDALGO COLOMBIA SAS
1,BD9/OUT/07176,46800.0,1625202,3.0,[DCE00937189] G937 FILTRO COMBUSTIBLE TOYOTA H...,FCAL5717,2025-10-01 14:07:04,FCAL5717,440706,SUCURSAL CALI,CLAUDIA ORTIZ,SUCURSALES / CALI / MOSTRADOR CALI CREDITO,35670,[CO9003546375] HIDALGO E HIDALGO COLOMBIA SAS
2,BD9/OUT/07176,56100.0,1625203,3.0,[DAE01127189] GA1127 FILTRO AIRE TOYOTA HILUX ...,FCAL5717,2025-10-01 14:07:04,FCAL5717,440706,SUCURSAL CALI,CLAUDIA ORTIZ,SUCURSALES / CALI / MOSTRADOR CALI CREDITO,35670,[CO9003546375] HIDALGO E HIDALGO COLOMBIA SAS
3,False,26500.0,1625200,1.0,"[BLS00189125] GS189 FILTRO ACEITE HYUNDAI, SAN...",PV3E20025 Mostrador Valladolid T2/15371,2025-10-01 14:05:04,PV3E20025,440705,SUCURSAL VALLADOLID,SEBAS RODRIGUEZ,SUCURSALES / Ventas Mostrador Valladolid,3912,[CO9006470826] GL TRANSPORTE SUMINISTROS Y AGR...
4,False,45200.0,1625199,1.0,[BCS00252125] GS252 FILTRO COMBUSTIBLE BOBCAT ...,PV3E20025 Mostrador Valladolid T2/15371,2025-10-01 14:05:04,PV3E20025,440705,SUCURSAL VALLADOLID,SEBAS RODRIGUEZ,SUCURSALES / Ventas Mostrador Valladolid,3912,[CO9006470826] GL TRANSPORTE SUMINISTROS Y AGR...


# EDA

## Facturas

In [3]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)




#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])


df = df.drop(columns=["origin"])

df_externos_bogota = df[(df["store_name"]=="PRINCIPAL COTA") & (df["Sucursal_tipo"]=="VENDEDOR EXTERNO")]

df = df.drop(df_externos_bogota.index)


In [4]:
df.groupby(["store_name","Bodega","Sucursal_tipo"])["quantity"].sum()

store_name             Bodega                             Sucursal_tipo   
PRINCIPAL COTA         PRINCIPAL COTA                     MOSTRADOR           15777.0
SUCURSAL BARRANQUILLA  BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO      113.0
                       SUCURSAL BARRANQUILLA              MOSTRADOR           24130.0
                                                          VENDEDOR EXTERNO    74035.0
                       SUCURSAL CALI                      VENDEDOR EXTERNO        2.0
SUCURSAL BUCARAMANGA    SUCURSAL BUCARAGAMNGA             VENDEDOR EXTERNO    46130.0
                       PRINCIPAL COTA                     VENDEDOR EXTERNO        2.0
                       SUCURSAL BUCARAMANGA               MOSTRADOR           18276.0
SUCURSAL CALI          BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO     2043.0
                       EXP.REP.DOMINICANA                 VENDEDOR EXTERNO     1961.0
                       PRINCIPAL COTA                     VENDEDO

In [5]:
df[df["product_name"].str.contains("DCE00833189")].sort_values(by=["quantity"],ascending=False)

,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv,Sucursal_tipo,origen,Bodega
12259,715000.0,1611056,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E43208 Mostrador Calle 6 T2/21164,2025-09-10 17:49:58,PV2E43208,436887,SUCURSAL CALLE 6,JAIME ANDRES CABALLERO CABALLERO,SUCURSALES / Ventas Mostrador Cll6,26064,[CC7334049] JAVIER SANCHEZ MONROY,MOSTRADOR,<NA>,SUCURSAL CALLE 6
101820,715000.0,1504546,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FBAQ13002,2025-04-02 21:04:23,FBAQ13002,408839,SUCURSAL BARRANQUILLA,MIGUEL ANGEL NIEBLES CERA,SUCURSALES / BARRANQUILLA / EXTERNOS,39889,[CO9012756624] ALMACEN Y TALLER TODO NISSAN A&...,VENDEDOR EXTERNO,BD6,SUCURSAL BARRANQUILLA
192810,514800.0,1395591,36.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV1E8545 Mostrador Cota T2/1936,2024-10-15 20:33:42,PV1E8545,380319,PRINCIPAL COTA,YURI NATALIA GAONA,SUCURSALES / Ventas Mostrador Cota,31909,[CO9015322728] REPUESTOS PINEDA SAS,MOSTRADOR,<NA>,PRINCIPAL COTA
58313,477000.0,1555318,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E40849 Mostrador Calle 6 T3/22423,2025-06-20 14:02:51,PV2E40849,422406,SUCURSAL CALLE 6,Camila Arguello,SUCURSALES / Ventas Mostrador Cll6,5062,[CO8300538946] TRANSCOLTUR S.A.S.,MOSTRADOR,<NA>,SUCURSAL CALLE 6
120589,540000.0,1481402,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E37805 Mostrador Calle 6 T3/21299,2025-02-28 14:32:09,PV2E37805,402900,SUCURSAL CALLE 6,Camila Arguello,SUCURSALES / Ventas Mostrador Cll6,5062,[CO8300538946] TRANSCOLTUR S.A.S.,MOSTRADOR,<NA>,SUCURSAL CALLE 6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,18000.0,1622165,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E43639 Mostrador Calle 6 T3/23472,2025-09-25 18:17:22,PV2E43639,439842,SUCURSAL CALLE 6,Camila Arguello,SUCURSALES / Ventas Mostrador Cll6,5062,[CO8300538946] TRANSCOLTUR S.A.S.,MOSTRADOR,<NA>,SUCURSAL CALLE 6
2731,18000.0,1622060,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV4E16260 Mostrador Norte T1/17950,2025-09-25 16:44:09,PV4E16260,439802,SUCURSAL NORTE,WILMER MULATO,SUCURSALES / Ventas Mostrador Norte,45311,[CO10306244699] JOSMMAN DAVID CHON RODRIGUEZ,MOSTRADOR,<NA>,SUCURSAL NORTE
2749,18000.0,1622039,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV9E1714 Mostrador Cali T1/2138,2025-09-25 16:15:04,PV9E1714,439791,SUCURSAL CALI,CLAUDIA ORTIZ,SUCURSALES / CALI / MOSTRADOR CALI CONTADO,41729,[CO10073604408] ESTEFANIA RESTREPO SUAREZ,MOSTRADOR,<NA>,SUCURSAL CALI
5173,18000.0,1619415,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV3E19920 Mostrador Valladolid T2/15299,2025-09-22 21:13:48,PV3E19920,439176,SUCURSAL VALLADOLID,SEBAS RODRIGUEZ,SUCURSALES / Ventas Mostrador Valladolid,27638,[CC222222222223] CONSUMIDOR FINAL VALLADOLID,MOSTRADOR,<NA>,SUCURSAL VALLADOLID


In [6]:
df_externos_bogota = df[(df["store_name"]=="SUCURSAL CALI") & (df["Bodega"]=="EXP.REP.DOMINICANA")]

df_externos_bogota

,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv,Sucursal_tipo,origen,Bodega
61149,3393750.0,1552174,125.0,[DAB02788025] DA2788 FILTRO AIRE DONSSON - Ch...,FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
61150,2062500.0,1552175,125.0,"[DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITAC...",FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
61151,1992000.0,1552183,24.0,[DAB02524025] DA2524 FILTRO AIRE - GMC BRIGADI...,FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
61152,204000.0,1552242,4.0,[DAB09027025] DA9027 FILTRO AIRE 2o. ATLAS COP...,FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
61153,150600.0,1552238,2.0,[DAR08183025] DA8183 FILTRO AIRE GENERADOR CUM...,FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61221,360000.0,1552193,24.0,"[DAB04671025] DA4671 FILTRO AIRE 2_ KUBOTA, NI...",FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
61222,91000.0,1552207,1.0,[DAR02106025] DA2106 FILTRO AIRE 1o. ISUZU 580...,FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
61223,765600.0,1552214,12.0,[DAB02805025] DA2805 FILTRO AIRE 1_ PERKINS PL...,FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
61224,571200.0,1552204,12.0,[DAR02005025] DA2005 FILTRO AIRE FOTON dob cab...,FCAL4648,2025-06-16 14:09:12,FCAL4648,421545,SUCURSAL CALI,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES / CALI / EXTERNOS CALI,3053,[CO8000120803] MUNDIAL DE FILTROS Y ACEITES S.A.S,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA


# UNION

## EMA SEMANAL CON VP SEMANALES (SOLO 2025)

In [7]:
# ===============================
# Procesar ventas normales
# ===============================
df_sales = df.copy()
df_sales["date_invoice"] = pd.to_datetime(df_sales["date_invoice"], errors="coerce")


# Referncia de producto
df_sales["product_ref"] = df_sales["product_name"].str.extract(r"\[([A-Z0-9]+)\]")


# Columnas temporales igual que en df_vp
df_sales["Semana"] = df_sales["date_invoice"].dt.to_period("W").dt.start_time
df_sales["ano"]    = df_sales["Semana"].dt.year
df_sales["mes"]    = df_sales["Semana"].dt.month
df_sales["dia"]    = df_sales["Semana"].dt.day

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
sales_by_week = (
    df_sales.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"], as_index=False)
    .agg(
        sales=("quantity", "sum"),              # suma de unidades vendidas
        num_facturas=("date_invoice", "nunique")  # cantidad de facturas distintas
    )
)


# Resultado
sales_by_week.sample(10)



,store_name,product_ref,Semana,ano,mes,dia,sales,num_facturas
57505,SUCURSAL CALLE 6,DAB04728025,2025-09-01,2025,9,1,2.0,1
84160,SUCURSAL NORTE,DAB02598025,2025-03-17,2025,3,17,1.0,1
51262,SUCURSAL CALLE 6,BCS10035125,2025-08-18,2025,8,18,2.0,1
81692,SUCURSAL NORTE,BCS00427125,2024-12-23,2024,12,23,4.0,1
48313,SUCURSAL CALLE 6,BCE00764125,2025-04-21,2025,4,21,2.0,2
80068,SUCURSAL NORTE,ALQ05W30116,2024-11-25,2024,11,25,1.0,1
81623,SUCURSAL NORTE,BCS00338125,2025-01-20,2025,1,20,1.0,1
38952,SUCURSAL CALI,BCS00412125,2025-04-07,2025,4,7,1.0,1
36176,SUCURSAL BUCARAMANGA,DCE00662189,2025-07-28,2025,7,28,12.0,2
41933,SUCURSAL CALI,DAB02524025,2025-07-21,2025,7,21,1.0,1


In [8]:
df_merged = pd.merge(
    sales_by_week[["store_name", "product_ref", "Semana", "sales","num_facturas"]],
    vp_week[["store_name", "product_ref", "Semana", "lost_sales","veces_vp"]],
    on=["store_name", "product_ref", "Semana"],
    how="outer"
).fillna(0)


In [9]:
df_merged["año"]    = df_merged["Semana"].dt.year
df_merged["mes"]    = df_merged["Semana"].dt.month
df_merged["dia"]    = df_merged["Semana"].dt.day


df_merged.head(10)

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
0,PRINCIPAL COTA,AC10388020,2025-09-22,0.0,0.0,36.0,4.0,2025,9,22
1,PRINCIPAL COTA,AGB0GRAS030,2025-07-28,0.0,0.0,1.0,1.0,2025,7,28
2,PRINCIPAL COTA,AHB0TO30132,2025-01-13,0.0,0.0,1.0,1.0,2025,1,13
3,PRINCIPAL COTA,AHB80W90030,2025-07-07,0.0,0.0,1.0,1.0,2025,7,7
4,PRINCIPAL COTA,AHB80W90050,2025-08-11,0.0,0.0,1.0,1.0,2025,8,11
5,PRINCIPAL COTA,AHBNTO68030,2024-10-21,1.0,1.0,0.0,0.0,2024,10,21
6,PRINCIPAL COTA,AHBNTO68030,2025-01-27,1.0,1.0,1.0,1.0,2025,1,27
7,PRINCIPAL COTA,AHBNTO68030,2025-02-03,0.0,0.0,1.0,1.0,2025,2,3
8,PRINCIPAL COTA,AHBNTO68030,2025-02-10,0.0,0.0,1.0,1.0,2025,2,10
9,PRINCIPAL COTA,AHBNTO68030,2025-02-17,0.0,0.0,4.0,1.0,2025,2,17


In [10]:
df_merged.query("product_ref == 'BAE05001125'")

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
15327,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-20,2.0,1.0,0.0,0.0,2025,1,20
15328,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24
15329,SUCURSAL BARRANQUILLA,BAE05001125,2025-03-24,0.0,0.0,3.0,3.0,2025,3,24
15330,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-08,1.0,1.0,0.0,0.0,2025,9,8
92722,SUCURSAL MEDELLIN,BAE05001125,2025-02-17,0.0,0.0,1.0,1.0,2025,2,17
92723,SUCURSAL MEDELLIN,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24
92724,SUCURSAL MEDELLIN,BAE05001125,2025-06-09,0.0,0.0,1.0,1.0,2025,6,9
92725,SUCURSAL MEDELLIN,BAE05001125,2025-08-18,0.0,0.0,1.0,1.0,2025,8,18
112336,SUCURSAL NORTE,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24


In [11]:
df_merged["product_ref"].nunique()

4309

## NORMALIZAR DF DE COSTOS

In [12]:
df_p["product_ref"] = df_p["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df_p_unique = (
    df_p[["product_ref", "producto_costo_unitario"]]
    .drop_duplicates(subset=["product_ref"])
)



df_p["product_ref"].nunique()

4835

## EMA 

In [13]:
import pandas as pd
import numpy as np

def compute_demand_and_ema(df,
                           alpha=0.20,        # peso de EMA
                           n_init=12,   # semanas o meses para inicializar EMA
                           date_col="Semana", #Semana o Mes
                           sales_col="sales",
                           lost_col="lost_sales",
                           freq ="W-MON" # "W-MON" semanas y "M" meses
                           ):
    df = df.copy()

    # ---------- Asegurar tipos y semana iniciando lunes ----------
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df[date_col] = df[date_col].dt.to_period(freq).dt.start_time #W_MON para lunes en semanas M para primer dia del mes

    # Asegurar numéricos
    df[sales_col] = pd.to_numeric(df[sales_col], errors="coerce").fillna(0)
    df[lost_col]  = pd.to_numeric(df[lost_col], errors="coerce").fillna(0)

    # ---------- Crear combinaciones completas ----------
    stores   = df["store_name"].unique()
    products = df["product_ref"].unique()


    # Crear rango completo desde min hasta max semana
    weeks = pd.period_range(df[date_col].min(), df[date_col].max(), freq=freq).to_timestamp()


    full_index = pd.MultiIndex.from_product([stores, products, weeks],
                                            names=["store_name", "product_ref", date_col])

    df = df.set_index(["store_name", "product_ref", date_col]).reindex(full_index).reset_index()

    # Rellenar ventas y perdidas con 0 en combinaciones faltantes
    df[sales_col] = df[sales_col].fillna(0)
    df[lost_col]  = df[lost_col].fillna(0)

    # Ordenar
    df = df.sort_values(["store_name", "product_ref", date_col])

    out_groups = []

    # ---------- Calcular demanda ajustada y EMA ----------
    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(date_col).reset_index(drop=True)
        sales = g[sales_col].to_numpy(dtype=float)
        lost  = g[lost_col].to_numpy(dtype=float)

        L = len(g)
        demanda = np.zeros(L, dtype=float)
        ema_arr = np.zeros(L, dtype=float)

        if L == 0:
            out_groups.append(g)
            continue

        # Inicialización EMA: promedio de primeras n_init ventas
        init_n = min(n_init, L)
        ema_prev = float(np.nanmean(sales[:init_n])) if init_n > 0 else 0.0
        if np.isnan(ema_prev):
            ema_prev = 0.0

        for i in range(L):
            s = sales[i]
            l = lost[i]

            # Regla 1
            if s >= 2.0 * l:
                demand_candidate = s + l
                demand = min(demand_candidate, 1.5 * s) if s > 0 else demand_candidate
            else:
                # Regla 2
                demand = s + 0.5 * ema_prev

            # Guardar redondeando
            demanda[i] = round(demand, 6)

            # Calcular EMA
            ema = alpha * demand + (1.0 - alpha) * ema_prev
            ema = round(ema, 6)
            ema_arr[i] = ema

            # actualizar para siguiente
            ema_prev = ema

        # Añadir columnas al grupo
        g = g.copy()
        g["demanda_ajustada"] = demanda
        g["EMA"] = ema_arr

        out_groups.append(g)

        

    # Concat resultados
    result = pd.concat(out_groups, ignore_index=True, sort=False)
    result = result.sort_values(["store_name", "product_ref", date_col]).reset_index(drop=True)
    return result

# ------------------ USO ------------------
df_with_demand1 = compute_demand_and_ema(df_merged, alpha=0.2, n_init=12) #estaba en 16

#APROX3min

In [14]:
df_filtrado_ordenado = df_with_demand1.query(
    "product_ref == 'DCE10126138' and store_name == 'SUCURSAL VALLADOLID'"
).sort_values(
    by=["Semana", "EMA"],  # Ordena primero por 'Semana', luego por 'EMA'
    ascending=[True, True] # Ambos en orden ascendente
)

df_filtrado_ordenado.tail(10)

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia,demanda_ajustada,EMA
1797962,SUCURSAL VALLADOLID,DCE10126138,2025-07-22,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.537746
1797963,SUCURSAL VALLADOLID,DCE10126138,2025-07-29,0.0,0.0,121.0,3.0,2025.0,8.0,4.0,0.268873,0.483971
1797964,SUCURSAL VALLADOLID,DCE10126138,2025-08-05,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.387177
1797965,SUCURSAL VALLADOLID,DCE10126138,2025-08-12,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.309742
1797966,SUCURSAL VALLADOLID,DCE10126138,2025-08-19,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.247794
1797967,SUCURSAL VALLADOLID,DCE10126138,2025-08-26,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.198235
1797968,SUCURSAL VALLADOLID,DCE10126138,2025-09-02,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.158588
1797969,SUCURSAL VALLADOLID,DCE10126138,2025-09-09,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.126870
1797970,SUCURSAL VALLADOLID,DCE10126138,2025-09-16,0.0,0.0,82.0,1.0,2025.0,9.0,22.0,0.063435,0.114183
1797971,SUCURSAL VALLADOLID,DCE10126138,2025-09-23,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.091346


In [15]:
def add_deviation_and_classification(df, ema_col="EMA", sales_col="sales", week_col="Semana", n_weeks=12): ##PERILLA SEMANAS 
    df = df.copy()
    out_groups = []

    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        
        sales = g[sales_col].to_numpy(dtype=float)
        ema   = g[ema_col].to_numpy(dtype=float)
        L = len(sales)

        # Inicializamos columnas
        g["desviacion_ema%"] = 0.0
        g["coef_ventas"] = 0.0

        if L < 2:
            out_groups.append(g)
            continue

        for i in range(L):
            # Usar solo hasta la semana actual (i)
            window_start = max(0, i - n_weeks)       # ventana de 12 semanas
            window_sales = sales[window_start:i]     # semanas previas
            window_ema   = ema[window_start:i]

            if len(window_sales) < 2:
                continue

            promedio_prev = np.mean(window_sales)
            if promedio_prev == 0:
                promedio_prev = 1e-6

            # --- Desviación usando EMAs previos ---
            desv = np.std(window_ema, ddof=1) / promedio_prev if promedio_prev > 0 else 0

            # --- Coeficiente de ventas ---
            coef = (max(window_sales) - min(window_sales)) / promedio_prev if promedio_prev > 0 else 0

            g.at[i, "desviacion_ema%"] = round(desv,2)
            g.at[i, "coef_ventas"] = coef

        out_groups.append(g)

    return pd.concat(out_groups, ignore_index=True)





df_with_demand2 = add_deviation_and_classification(df_with_demand1)
df_with_demand2 = df_with_demand2[["store_name","product_ref","Semana","num_facturas","sales","lost_sales","demanda_ajustada","EMA","desviacion_ema%","coef_ventas"]]

# Demora mas o menos 8 min

In [16]:
df_with_demand2.query("product_ref == 'BAE05001125'").sort_values(by=["EMA"]).tail(10)

,store_name,product_ref,Semana,num_facturas,sales,lost_sales,demanda_ajustada,EMA,desviacion_ema%,coef_ventas
252726,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-25,NaN,0.0,0.0,0.00000,0.117965,0.87,12.0
252755,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-16,NaN,0.0,0.0,0.00000,0.128206,0.84,12.0
252725,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-18,0.0,0.0,1.0,0.08192,0.147456,0.90,12.0
252754,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-09,NaN,0.0,0.0,0.00000,0.160257,0.69,12.0
252724,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-11,NaN,0.0,0.0,0.00000,0.163840,0.91,12.0
252753,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-02,1.0,1.0,0.0,1.00000,0.200321,1378.24,0.0
252723,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-04,NaN,0.0,0.0,0.00000,0.204800,0.90,12.0
252722,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-28,NaN,0.0,0.0,0.00000,0.256000,0.85,12.0
252721,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-21,NaN,0.0,0.0,0.00000,0.320000,0.69,12.0
252720,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-14,1.0,2.0,0.0,2.00000,0.400000,0.00,0.0


In [17]:
df_with_demand2["semana_num"] = df_with_demand2["Semana"].dt.isocalendar().week
df_with_demand2["año"] = df_with_demand2["Semana"].dt.year


### AGREGAR SEMANA CSV

In [18]:
df_demand_2025 = df_with_demand2[df_with_demand2["año"]==2025] #Ajustar año

demand_2025 = df_demand_2025[
    ["store_name","product_ref","año","Semana","semana_num",
     "num_facturas","EMA","demanda_ajustada","desviacion_ema%","coef_ventas","lost_sales"]
]


demand_2025_38 = demand_2025[demand_2025["semana_num"]==38]  #Ajustar semana numero

demand_2025_38.sort_values(by=["EMA"],ascending=False).tail(30)




,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales
707124,SUCURSAL CALI,BAE03444125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
707866,SUCURSAL CALI,BAE04106125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
710039,SUCURSAL CALI,BAE06235125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
709933,SUCURSAL CALI,BAE06004125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
709880,SUCURSAL CALI,BAE06003125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
709827,SUCURSAL CALI,BAE06001125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
709774,SUCURSAL CALI,BAE05447125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
709721,SUCURSAL CALI,BAE05171125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
709668,SUCURSAL CALI,BAE05091125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0
709615,SUCURSAL CALI,BAE05029125,2025,2025-09-16,38,NaN,0.0,0.0,0.0,0.0,0.0


## UNIR COSTO

In [19]:
df_merge_def = pd.merge(
    demand_2025_38,
    df_p_unique,
    on="product_ref",
    how="outer" #para incluir todos , estaba antes left
).fillna(0)

#df_merge_def.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202538.csv")

#merge_def = df_merge_def[df_merge_def["producto_costo_unitario"] !=0].copy() #Eliminar productos sin costos unitarios

merge_def =  df_merge_def.copy()
df_merge_def.sort_values(by=["EMA"],ascending=False).sample(10)

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales,producto_costo_unitario
10612,SUCURSAL CALI,BHE01052125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,0.00
10213,SUCURSAL BARRANQUILLA,BHE00865125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,14949.59
11376,SUCURSAL MEDELLIN,BHS00486125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,135739.24
16308,PRINCIPAL COTA,DAB08002025,2025.0,2025-09-16 00:00:00,38,0.0,0.000786,0.0,3372.37,0.0,0.0,24172.13
2709,SUCURSAL VALLADOLID,BAE01140125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,41111.00
15240,SUCURSAL CALLE 6,DAB02033025,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,57885.15
23619,SUCURSAL BARRANQUILLA,DAE06544115,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,7147.00
28353,SUCURSAL NORTE,DCE00125186,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,5775.68
515,SUCURSAL CALI,BAB02028125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,58794.31
5876,SUCURSAL BARRANQUILLA,BCE00805125,2025.0,2025-09-16 00:00:00,38,0.0,0.003734,0.0,16031.13,0.0,0.0,13916.12


In [20]:
merge_def[merge_def["store_name"]==0].sample(10)

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales,producto_costo_unitario
23580,0,DAE06388010,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4437.89
33115,0,DLE00001017,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.00
34148,0,DLS00077010,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,18983.85
34985,0,SERV10,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
28690,0,DCE00470189,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4336.73
9428,0,BHE00282125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,292610.00
285,0,ALG10W40030,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,59447.62
10254,0,BHE00876125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,111698.00
19840,0,DAE00699136,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,8783.76
26618,0,DAR02800025,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,13132.50


In [21]:
merge_def.query("product_ref == 'BAE05001125'")

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales,producto_costo_unitario
3730,PRINCIPAL COTA,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3731,SUCURSAL BARRANQUILLA,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.128206,0.0,0.84,12.0,0.0,90836.27
3732,SUCURSAL BUCARAMANGA,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3733,SUCURSAL CALI,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3734,SUCURSAL CALLE 6,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3735,SUCURSAL MEDELLIN,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3736,SUCURSAL NORTE,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3737,SUCURSAL VALLADOLID,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
